# NHIS/HIRA 국가 표준 의료 데이터 메타데이터 파서

> **Purpose:** 국민건강보험공단(NHIS) 및 건강보험심사평가원(HIRA)의 표준 메타데이터를 분석하여
> MCMC 합성 데이터 생성의 사전분포(Prior) 파라미터를 도출합니다.
>
> **참조:**
> - NHIS 표본 코호트 DB: https://www.data.go.kr/data/15007115/fileData.do
> - HIRA 보건의료빅데이터: https://opendata.hira.or.kr/
> - 건강검진정보: https://www.data.go.kr/data/15007122/fileData.do

## 1. 라이브러리 및 설정

In [ ]:

from pyspark.sql import functions as F
from pyspark.sql.types import *
import json

## 2. NHIS 표본 코호트 메타데이터 구조 정의

국민건강보험공단 표본 코호트 DB의 주요 필드를 정의합니다.

In [ ]:

# NHIS 표본 코호트 스키마 정의 (100만 명 표본)
nhis_schema = StructType([
    StructField("STND_Y", StringType(), True),          # 기준연도
    StructField("SEX_TYPE", StringType(), True),         # 성별코드 (1:남, 2:여)
    StructField("AGE_GROUP", StringType(), True),        # 연령대코드
    StructField("SIDO", StringType(), True),             # 시도코드
    StructField("MAIN_SICK", StringType(), True),        # 주상병코드 (KCD)
    StructField("DGRSV_CD", StringType(), True),         # 진료과목코드
    StructField("MDCARE_DD_CNT", IntegerType(), True),   # 요양일수
    StructField("TOT_PRSC_DD_CNT", IntegerType(), True), # 총처방일수
    StructField("EDEC_TRMT_AMT", DoubleType(), True),    # 심결요양급여비용총액
])

# HIRA 진료내역 스키마 정의
hira_schema = StructType([
    StructField("YM", StringType(), True),               # 진료년월
    StructField("SICK_SYM1", StringType(), True),        # 상병기호1
    StructField("SICK_SYM2", StringType(), True),        # 상병기호2
    StructField("OPRTN_CD", StringType(), True),         # 수술코드
    StructField("MDCN_CD", StringType(), True),          # 의약품코드
    StructField("MDCN_QTY", DoubleType(), True),         # 투약량
    StructField("MDCN_DD_CNT", IntegerType(), True),     # 투약일수
])

print("✅ NHIS/HIRA 스키마 정의 완료")

## 3. 건강검진 정보 메타데이터 구조

In [ ]:

# 건강검진 정보 스키마 (100만 명 표본)
health_checkup_schema = StructType([
    StructField("HEIGHT", DoubleType(), True),           # 신장(cm)
    StructField("WEIGHT", DoubleType(), True),           # 체중(kg)
    StructField("WAIST", DoubleType(), True),            # 허리둘레(cm)
    StructField("BP_HIGH", IntegerType(), True),         # 수축기혈압(mmHg)
    StructField("BP_LWST", IntegerType(), True),         # 이완기혈압(mmHg)
    StructField("BLDS", IntegerType(), True),            # 식전혈당(mg/dL)
    StructField("TOT_CHOLE", IntegerType(), True),       # 총콜레스테롤(mg/dL)
    StructField("TRIGLYCERIDE", IntegerType(), True),    # 중성지방(mg/dL)
    StructField("HDL_CHOLE", IntegerType(), True),       # HDL콜레스테롤(mg/dL)
    StructField("LDL_CHOLE", IntegerType(), True),       # LDL콜레스테롤(mg/dL)
    StructField("HMG", DoubleType(), True),              # 혈색소(g/dL)
    StructField("OLIG_PROTE_CD", StringType(), True),    # 요단백
    StructField("CREATININE", DoubleType(), True),       # 혈청크레아티닌(mg/dL)
    StructField("SGOT_AST", IntegerType(), True),        # AST(SGOT)
    StructField("SGPT_ALT", IntegerType(), True),        # ALT(SGPT)
    StructField("GAMMA_GTP", IntegerType(), True),       # 감마지티피
    StructField("SMK_STAT_TYPE_CD", StringType(), True), # 흡연상태코드
    StructField("DRK_YN", StringType(), True),           # 음주여부
])

print("✅ 건강검진 스키마 정의 완료")

## 4. 메타데이터 기반 통계량 산출

실제 데이터 대신 공개된 통계량을 활용하여 MCMC 사전분포 파라미터를 산출합니다.

In [ ]:

# 국민건강보험통계연보 기반 통계량 (2023년 기준)
population_statistics = {
    "demographics": {
        "total_population": 51_740_000,
        "age_distribution": {
            "0-19":  0.17,
            "20-39": 0.25,
            "40-59": 0.30,
            "60-79": 0.22,
            "80+":   0.06,
        },
        "gender_ratio": {"M": 0.4997, "F": 0.5003},
    },
    "vital_signs_reference": {
        "BP_systolic":   {"mean": 121.3, "std": 15.2, "p5": 98, "p95": 148},
        "BP_diastolic":  {"mean": 76.1,  "std": 10.8, "p5": 60, "p95": 95},
        "blood_glucose": {"mean": 100.2, "std": 28.5, "p5": 76, "p95": 140},
        "total_cholesterol": {"mean": 196.4, "std": 37.1, "p5": 140, "p95": 260},
        "BMI":           {"mean": 24.3, "std": 3.5, "p5": 18.5, "p95": 31.0},
        "hemoglobin":    {"mean": 14.1, "std": 1.6, "p5": 11.5, "p95": 16.8},
    },
    "disease_prevalence": {
        "hypertension":       0.284,  # 고혈압
        "diabetes":           0.139,  # 당뇨
        "hyperlipidemia":     0.227,  # 이상지질혈증
        "ischemic_heart":     0.042,  # 허혈성심장질환
        "cerebrovascular":    0.035,  # 뇌혈관질환
        "respiratory":        0.052,  # 호흡기질환
    }
}

print("✅ 국가 통계량 기반 사전분포 파라미터 산출 완료")
print(json.dumps(population_statistics["vital_signs_reference"], indent=2))

## 5. 결과 저장 (MCMC Generator 연동용)

In [ ]:

# 사전분포 파라미터를 JSON으로 저장
prior_params = {
    "demographics": population_statistics["demographics"],
    "vital_signs": population_statistics["vital_signs_reference"],
    "disease_prevalence": population_statistics["disease_prevalence"],
    "data_sources": [
        "NHIS 표본 코호트 DB (100만 명)",
        "HIRA 보건의료빅데이터",
        "국민건강보험공단 건강검진정보",
    ]
}

# Spark DataFrame으로 변환하여 Delta Lake 저장
df_priors = spark.createDataFrame([{
    "param_name": "mcmc_prior_v1",
    "params_json": json.dumps(prior_params, ensure_ascii=False),
    "created_at": F.current_timestamp()
}])

# 메타데이터 테이블에 저장 (Bronze Layer)
# df_priors.write.mode("overwrite").saveAsTable("bronze.mcmc_prior_parameters")

print("✅ MCMC 사전분포 파라미터 준비 완료")
print("→ 01_MCMC_SyntheticData_Generator.py에서 이 파라미터를 사용합니다.")